# Calculate the surface terriere

In [13]:
# Calculate the surface terriere
import math

# Define the diameter
diameter = 63.00	

# Define pi (you can use the built-in constant or approximate value)
pi = math.pi

# Calculate the surface terriere (without rounding)
surface_terriere = (diameter**2) / (4 * pi)

# Print the result
print(round(surface_terriere, 2))


315.84


In [16]:
import pymysql.cursors
import pandas as pd
import matplotlib.pyplot as plt

# Establish your database connection
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             database='liege1',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Get the selected option value from the request body
        option = input('Enter the option: ')

        # Define the base query
        query = """
            SELECT 
                id, 
                serie, 
                CONCAT(parcelle, sous_parcelle) AS Sousparcelle, 
                code_wilaya, 
                commune, 
                total_diametre, 
                average_diametre, 
                nombre_tree, 
                total_hauteur_total, 
                average_hauteur_total, 
                total_age, 
                average_age,
                ROUND((POW(total_diametre, 2) / (4 * PI())),2) AS surface_terriere
            FROM (
                SELECT 
                    sp.id, 
                    sp.serie, 
                    sp.parcelle, 
                    sp.sous_parcelle,
                    sp.code_wilaya, 
                    sp.commune, 
                    ROUND(SUM(arbres.diametre), 2) AS total_diametre, 
                    ROUND(AVG(arbres.diametre), 2) AS average_diametre, 
                    COUNT(arbres.id_arbre) AS nombre_tree, 
                    ROUND(SUM(arbres.hauteur_total), 2) AS total_hauteur_total, 
                    ROUND(AVG(arbres.hauteur_total), 2) AS average_hauteur_total, 
                    ROUND(SUM(arbres.hauteur_demasclage), 2) AS total_hauteur_demasclage, 
                    ROUND(AVG(arbres.hauteur_demasclage), 2) AS average_hauteur_demasclage, 
                    ROUND(SUM(
                        CASE 
                            WHEN sp.code_wilaya = '18' THEN ROUND((arbres.epaisseur * 10) / 3.14, 2)
                            WHEN sp.code_wilaya = '6' THEN ROUND((arbres.epaisseur * 10) / 3.19, 2)
                            WHEN sp.code_wilaya = '13' THEN ROUND((arbres.epaisseur * 10) / 2.64, 2)
                            ELSE NULL  
                        END
                    ), 2) AS total_age, 
                    ROUND(AVG(
                        CASE 
                            WHEN sp.code_wilaya = '18' THEN ROUND((arbres.epaisseur * 10) / 3.14, 2)
                            WHEN sp.code_wilaya = '6' THEN ROUND((arbres.epaisseur * 10) / 3.19, 2)
                            WHEN sp.code_wilaya = '13' THEN ROUND((arbres.epaisseur * 10) / 2.64, 2)
                            ELSE NULL  
                        END
                    ), 2) AS average_age,
                    arbres.age
                FROM 
                    sp
                JOIN 
                    points ON sp.id = points.id_sp
                JOIN 
                    arbres ON points.id_point = arbres.id_point 
                WHERE 
                    sp.exist = 1 AND points.exist = 1 AND arbres.exist = 1
        """

        # Adjust the query based on the selected option
        if option == '1':
            query += " GROUP BY sp.id) AS subquery ORDER BY id ASC"
        elif option == '2':
            query += " GROUP BY points.id_point) AS subquery ORDER BY id ASC"
        elif option == '3':
            query += " GROUP BY arbres.id_arbre) AS subquery ORDER BY id ASC"
        else:
            query += " GROUP BY sp.id) AS subquery ORDER BY id ASC"

        # Execute the query
        cursor.execute(query)
        result = cursor.fetchall()

        # Convert the result into a pandas DataFrame
        df = pd.DataFrame(result)

        # Display the DataFrame as a table
        display(df)

        # Create a bar chart
        df.plot(kind='bar', x='id', y='total_diametre')
        plt.xlabel('ID')
        plt.ylabel('Total Diameter')
        plt.title('Total Diameter by ID')
        plt.show()

finally:
    connection.close()


ModuleNotFoundError: No module named 'pymysql'